In [1]:
import imutils
import numpy as np
import cv2

In [2]:
from tensorflow.keras.models import load_model
model1 = load_model('model_mask_final.h5')
model2 = load_model('model_emotions_final.h5')

In [3]:
print("[INFO] loading model...")
prototxt = 'deploy.prototxt'
model = 'res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt, model)

[INFO] loading model...


In [5]:
import cv2

cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()

    if ret:

        frame = imutils.resize(frame, width=600)
        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                text = "{:.2f}%".format(confidence * 100)
                y = startY - 10 if startY - 10 > 10 else startY + 10
                
                minX, maxX = min(startX, endX), max(startX, endX)
                minY, maxY = min(startY, endY), max(startY, endY)
                face = frame[minY:maxY, minX:maxX].copy()
                
                img = cv2.resize(face,(50,50))
                img = np.array([img/255.])
                y_hat = model1.predict(img)
                y_hat = np.argmax(y_hat)
                mask = ('Nomask', 'Mask', 'Incorect')
                predicted_mask = mask[y_hat]
                cv2.putText(frame,predicted_mask, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 255), 2) 

                
                img = cv2.resize(face,(48,48))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img = np.array([img/255.])
                emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
                y_hat2 = model2.predict(img)
                y_hat2 = np.argmax(y_hat2)
                
                predicted_emotion = emotions[y_hat2]
                

                # predicted_emotion = emotions[y_hat2]
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 0, 255), 2)
                if y_hat == 0:
                    
                    #cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 0, 255), 2)
                    cv2.putText(frame,predicted_emotion, (endX-10,endX),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2) 
        cv2.imshow("frame", frame)
        
        key = cv2.waitKey(1)

        if key == 27:

            break

 

cap.release()

cv2.destroyAllWindows()

1/1 [==============================] - 0s 73ms/step


KeyboardInterrupt: 